# Import Library

In [1]:
import pandas as pd
import numpy as np

import selenium
from selenium import webdriver

from bs4 import BeautifulSoup as bs
from urllib.request import urlopen, Request
from urllib.parse import quote_plus

import time
from datetime import datetime
from tqdm import tqdm_notebook
import warnings
warnings.filterwarnings('ignore')

# Create Empty List & Parameter Setting

In [2]:
url = 'http://www.instagram.com'
path = 'C:\\Users\\rectworks\\Downloads\\chromedriver_win32\\chromedriver.exe'

SCROLL_PAUSE_TIME = 3.0
ID = 'hanshow113'
PW = 'wlsldjtm12!'

post_link = []
popularPost_len = []

id_list = []
like_list = []
tag_list = []
link_list = []
date_list = []
month_list = []
day_list = []


# Define Function

In [3]:
def instagram_login(id, pw):
    driver.get(url)
    driver.implicitly_wait(5)
    driver.find_element_by_name('username').send_keys(id)    # id 입력
    elem_pw = driver.find_element_by_name('password')    # pw 입력
    elem_pw.send_keys(pw)
    elem_pw.submit()
    
    driver.implicitly_wait(5)    # 파싱될 때까지 5초 기다림 (미리 완료되면 waiting 종료됨)
    driver.find_element_by_class_name('cmbtv').click()    # 비밀번호 저장하지 않음
    
    driver.implicitly_wait(5)
    driver.find_element_by_xpath('/html/body/div[4]/div/div/div/div[3]/button[2]').click()    # 알림설정 무시

In [4]:
def main_search(keyword):
    search = driver.find_element_by_xpath('//*[@id="react-root"]/section/nav/div[2]/div/div/div[2]/input')
    search.clear()
    search.send_keys(keyword)
    search_list1 = driver.find_element_by_xpath('//*[@id="react-root"]/section/nav/div[2]/div/div/div[2]/div[3]/div[2]/div/a[1]')
    search_list1.click()

In [5]:
def scroll_page():
    post_link.clear()
    popularPost_len.clear()
    while True:
        pageString = driver.page_source   # page_source : 현재 렌더링된 페이지의 Elements를 모두 가져옴
        bsObj = bs(pageString, 'lxml')

        for postline in bsObj.find_all(name='div', attrs={"class":"Nnq7C weEfm"}):
            a_len = len(postline.select('a'))
            popularPost_len.append(a_len)
            # 인스타그램 게시물은 행별로 최대 3개까지 확인할 수 있는데, 최근게시물이나 마지막 게시물은 1,2개가 나올 수도 있어서 len 지정
            for post in range(a_len):   
                item = postline.select('a')[post]
                link = item.attrs['href']
                if link not in post_link:   # 스크롤을 내리고 중복된 것을 제거하지 않고 누적시키기 때문에 없는 것만 추가
                    post_link.append(link)

        last_height = driver.execute_script('return document.body.scrollHeight')   # 자바스크림트로 스크롤 길이를 넘겨줌
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")   # selenium에서 scroll 기능 사용
        time.sleep(SCROLL_PAUSE_TIME)
        # 프로세스 자체를 지정시간동안 기다려줌(무조건 지연)
        #driver.implicitly_wait(SCROLL_PAUSE_TIME)
        # 브라우저 엔진에서 파싱되는 시간을 기다려줌(요소가 존재하면 지연없이 코드 실행)
        new_height = driver.execute_script("return document.body.scrollHeight")

        if new_height == last_height:
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(SCROLL_PAUSE_TIME)
            #driver.implicitly_wait(SCROLL_PAUSE_TIME)
            new_height = driver.execute_script("return document.body.scrollHeight")

            if new_height == last_height:
                break
            else:
                last_height = new_height
                continue

In [6]:
def popularPostSum():
    popularPost = 0
    for i in range(3):
        popularPost += popularPost_len[i]
    return popularPost

# 인기게시물은 시간순으로 나타나는 것이 아니므로
# 여기에 나타나는 항목들은 시간순으로 데이터프레임 만다는 데서 제외시키기 위한 함수

In [31]:
def to_timestamp(std_date):
    date_string = std_date
    date =datetime.strptime(date_string, "%Y.%m.%d")
    time_tuple = date.timetuple()
    std_timestamp = time.mktime(time_tuple)
    return int(std_timestamp)
    

In [79]:
def date_based_crawling(std_date):
    id_list.clear()
    like_list.clear()
    tag_list.clear()
    link_list.clear()
    date_list.clear()
    month_list.clear()
    day_list.clear()
    
    
    post_linkx = post_link[popularPostSum():]    # 스크롤 내리면서 모은 데이터 중 인기 게시물은 제외시킴
    num_of_postx = len(post_linkx)
    

    std_timestamp = to_timestamp(std_date)
    # 기준 날짜를 지나면 크롤링이 멈추도록 작동
    
    for i in tqdm_notebook(range(num_of_postx)):
        req = Request("https://www.instagram.com" + post_linkx[i], headers={'User-Agent': 'Mozila/5.0'})
        postpage = urlopen(req).read()
        

        post_body = bs(postpage, 'lxml', from_encoding='utf-8')
        post_core = post_body.find('meta', attrs={'property':"og:description"})
        contents = post_core['content']

        # break할 시간 비교
        posttxt = str(postpage)    
        timestamp = int(posttxt[posttxt.find('taken_at_timestamp')+20 : posttxt.find('taken_at_timestamp')+30])
        
        
        if (std_timestamp > timestamp) and (to_timestamp(date_list[-1][:10]) - timestamp) > 86400 : 
            pass
        if (std_timestamp > timestamp) and (to_timestamp(date_list[-1][:10]) - timestamp) < 86400 : 
            break
        # 게시글 중간에 오래된 게시글이 올라오는 경우가 있어서 이전 게시글과 하루 이상 차이(timestamp 86400)가 나면
        # outlier로 간주하고 pass
        
        
        # 시간
        date_list.append(datetime.fromtimestamp(timestamp).strftime('%Y.%m.%d %H:%M'))
        month_list.append(datetime.fromtimestamp(timestamp).strftime("%m"))
        day_list.append(datetime.fromtimestamp(timestamp).strftime("%d"))

        # 개별 링크 리스트 
        link_list.append("https://www.instagram.com" + post_linkx[i])   
        
        # 좋아요
        try:
            likes = int(contents[: contents.find(' Likes, ')])  # Likes 문자열 앞에 있는 좋아요 개수 추출
        except:
            likes=0  # 좋아요 가 아니라 조회수로 표시되는 경우도 있어 이런 경우는 0으로 표시
        like_list.append(likes)

        # 개별 계정
        if "@" and ")" in contents:
            personal_id = contents[contents.find("@")+1 : contents.find(")")]
        elif "shared a post on Instagram" in contents:
            personal_id = contents[contents.find("@")+1 : contents.find('shared a post on Instagram')]                
        elif "shared a photo on Instagram" in contents:
            personal_id = contents[contents.find("@")+1 : contents.find('shared a photo on Instagram')]                
        elif "@" and ")" not in contents and "on Instagram" in contents:
            personal_id = contents[contents.find("@")+1 : contents.find('on Instagram')]
    
        else :
            personal_id = contents[1: contents.find(' posted on')]
        id_list.append(personal_id)

        '''    
        (@personal_id) on instagram, @persoanlid posted on instagram, personal_id on instgram 등의 형태로 meta 데이터에 표시되기
        때문에 여러 형식별 id 추출 if문 수행
        '''

        # 해시태그
        tag_list.append([])
        for tag_content in post_body.find_all('meta', attrs={'property':"instapp:hashtags"}):
            hashtags = tag_content['content'].rstrip(',')
            tag_list[i].append(hashtags)

# Operation check

In [8]:
driver = webdriver.Chrome(path)

instagram_login(ID, PW)

In [9]:
main_search('#춘천맛집추천')

In [10]:
scroll_page()

KeyboardInterrupt: 

In [11]:
len(post_link)

1257

In [68]:
post_link[11]

'/p/CDHpD58hOvr/'

In [77]:
req = Request("https://www.instagram.com" + post_link[11], headers={'User-Agent': 'Mozila/5.0'})
postpage = urlopen(req).read()


post_body = bs(postpage, 'lxml', from_encoding='utf-8')
post_core = post_body.find('meta', attrs={'property':"og:description"})
contents = post_core['content']
contents

'이선화 shared a post on Instagram: “#황토집숯불닭갈비…” • Follow their account to see 1,396 posts.'

In [78]:
post_core

<meta content="이선화 shared a post on Instagram: “#황토집숯불닭갈비…” • Follow their account to see 1,396 posts." property="og:description"/>

In [83]:
date_based_crawling('2020.07.01')

In [84]:
insta_dict = {'날짜':date_list,'월':month_list,'일':day_list,'계정':id_list,
              '좋아요':like_list,'해시태그':tag_list,'링크':link_list}
df = pd.DataFrame(insta_dict)
df

,날짜,월,일,계정,좋아요,해시태그,링크
0,2020.07.27 09:36,07,27,kiiae89,2,"[숯불닭갈비, 소양강댐맛집, 춘천맛집, 춘천외식, 소양강, 소양강닭갈비, 춘천맛집추...",https://www.instagram.com/p/CDICoMMgno4/
1,2020.07.27 09:01,07,27,nmkyu93,6,"[숯불닭갈비, 여행, 춘천숯불닭갈비맛집, 춘천외식, 닭갈비맛집, 닭갈비, 춘천맛집황...",https://www.instagram.com/p/CDH-n-UluKE/
2,2020.07.27 05:53,07,27,이선화,0,"[춘천숯불닭갈비, 춘천맛집베스트, 황토집숯불닭갈비, 춘천닭갈비맛집, 춘천닭갈비존맛,...",https://www.instagram.com/p/CDHpD58hOvr/
3,2020.07.27 05:24,07,27,kihun_0111,5,"[춘천맛집황토집숯불닭갈비, 춘천맘, 소양강맛집, 춘천, 춘천여행추천, 춘천시, 춘천...",https://www.instagram.com/p/CDHl1JLFZox/
4,2020.07.27 04:46,07,27,92ppark,3,"[맛집투어, 춘천맛집추천, 소양강댐맛집, 닭갈비, 소양강닭갈비, 춘천닭갈비, 춘천여...",https://www.instagram.com/p/CDHhc3cFeOv/
...,...,...,...,...,...,...,...
1226,2020.07.01 04:32,07,01,jengmin5719,10,"[춘천맛집, 춘천맛집추천, 춘천닭갈비맛집, 춘천숯불닭갈비맛집, 춘천닭갈비, 춘천맛집...",https://www.instagram.com/p/CCEjHNql2kR/
1227,2020.07.01 03:51,07,01,j.sung_hun2019,7,"[춘천맛집황토집숯불닭갈비, 소양강맛집, 춘천닭갈비맛집, 춘천맛집추천, 춘천, 춘천외...",https://www.instagram.com/p/CCEedW_g5kX/
1228,2020.07.01 03:04,07,01,,0,"[춘천맛집추천, 춘천닭갈비, 춘천여행중, 소양강닭갈비, 춘천, 춘천닭갈비맛집, 소양...",https://www.instagram.com/p/CCEZGiZA6-Z/
1229,2020.07.01 02:03,07,01,3009_1995,10,"[춘천닭갈비, 춘천맛집황토집숯불닭갈비, 좋아요, 춘천맛집추천, 먹스타그램, 맛스타,...",https://www.instagram.com/p/CCESKo-nD2H/


In [85]:
df.sort_values(by=['좋아요'], axis=0, ascending=False)

,날짜,월,일,계정,좋아요,해시태그,링크
724,2020.07.12 11:44,07,12,chuncheonlike,475,"[춘천핫플, 춘천데이트코스, 춘라로드14, 춘천핫플레이스, 거두리, 춘천교대, 퇴계...",https://www.instagram.com/p/CChpVOnAzJD/
488,2020.07.16 23:31,07,16,chuncheonlike,465,"[백반맛집, 춘천맛집, 춘천시, 춘천, 강원대학교, 강대후문, 큰골집, 강원도맛집,...",https://www.instagram.com/p/CCtNe69Ag9A/
845,2020.07.09 17:36,07,09,bob_yeongnaring,457,"[누룽지탕, 춘천존맛, 한울타리, 춘천맛집, 소통해요, 선팔하면맞팔, 소통, 먹스타...",https://www.instagram.com/p/CCajQ_El_3X/
621,2020.07.14 16:25,07,14,___lidam,379,"[셀카스타그램, today, selstagram, ootd, 춘천맛집, dailyg...",https://www.instagram.com/p/CCnTGbtHIob/
737,2020.07.12 03:29,07,12,___lidam,370,"[selfies, 해바라기, 셀카, dessert, ootd, 춘천맛집, 얼스타그램...",https://www.instagram.com/p/CCgws85nHGP/
...,...,...,...,...,...,...,...
579,2020.07.15 12:49,07,15,forevershsen: “#Repost @chuncheonlike with @ge...,0,"[춘천데이트코스, 퇴계동, 춘천댁, 춘천여행, 춘천, 계백수제갈비_퇴계점, 춘천가볼...",https://www.instagram.com/p/CCpfKJUl1Wd/
574,2020.07.15 13:20,07,15,__petit_cochon_,0,"[춘천셀카맛집, 예스키즈존, 춘천마카롱맛집, 춘천맛집추천, 쁘띠꼬숑, 춘천디저트맛집...",https://www.instagram.com/p/CCpir8bpugA/
1059,2020.07.04 19:41,07,04,,0,"[춘천맛집, 춘천카페, 웨어커피, 춘천명동, 춘천명동카페, 춘천명동맛집, 춘천핫플,...",https://www.instagram.com/p/CCN5mVAjFZ3/
569,2020.07.15 15:31,07,15,오은혜,0,"[소양강, 소양강맛집, 춘천여행추천, 춘천맛집추천, 춘천닭갈비맛집, 맛스타그램, 춘...",https://www.instagram.com/p/CCpxwFYF7_U/
